In [1]:
import importlib

import pandas as pd
import dotenv
import os

dotenv.load_dotenv('.env')
MT5_SERVER = os.environ["MT5_SERVER"]
MT5_LOGIN = os.environ["MT5_LOGIN"]
MT5_PASSWORD = os.environ["MT5_PASSWORD"]
DATA_PATH = os.environ["DATA_PATH"]
CATALOG_PATH = os.path.join(os.getcwd(), os.environ["CATALOG_PATH"])

# nautilus_trader imports

from nautilus_trader.model.position import Position
from nautilus_trader.model.objects import Price
from nautilus_trader.model.identifiers import Venue, InstrumentId, Symbol
from nautilus_trader.model.data import Bar, BarType, QuoteTick
from nautilus_trader.config import BacktestVenueConfig, BacktestDataConfig, BacktestRunConfig, BacktestEngineConfig, RiskEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.engine import BacktestResult
from nautilus_trader.trading.strategy import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.core.datetime import dt_to_unix_nanos, maybe_unix_nanos_to_dt, unix_nanos_to_dt
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.cache.cache import Cache
from nautilus_trader.test_kit.providers import TestInstrumentProvider
from nautilus_trader.persistence.wranglers import QuoteTickDataWrangler
from nautilus_trader.persistence.wranglers import BarDataWrangler
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.persistence.loaders import ParquetBarDataLoader

import decimal

# other imports
from pandas import Timestamp
import importlib
import mplfinance as mpf
import matplotlib.pyplot as plt

# my packages
from put101 import indicators
from put101 import utils
from put101 import vizz

# ---------------- CONFIGURATION ----------------
catalog = ParquetDataCatalog(CATALOG_PATH)

venue_str = "SIM_EIGHTCAP"
venue = Venue(venue_str)

symbol_str = "EURUSD"
symbol = Symbol(symbol_str)
instrument_id_str = f"EURUSD.{venue}"
instrument_id = InstrumentId(symbol, venue)
instrument = TestInstrumentProvider.default_fx_ccy(symbol_str, venue)

bar_type=f"{instrument_id}-15-MINUTE-BID-INTERNAL",
print(bar_type)

('EURUSD.SIM_EIGHTCAP-15-MINUTE-BID-INTERNAL',)


In [2]:
# setup loaders
wrangler = QuoteTickDataWrangler(instrument)
catalog = ParquetDataCatalog(CATALOG_PATH)

In [3]:
# params
start = dt_to_unix_nanos(pd.Timestamp("2022-01-01 00:00:00"))
end = start + pd.Timedelta(days=30).value

In [4]:
ticks = catalog.quote_ticks(instrument_ids=[instrument.symbol.value], start=start, end=end)

In [5]:
t: QuoteTick = ticks[0]
QuoteTick.to_dict(t)

{'type': 'QuoteTick',
 'instrument_id': 'EURUSD.SIM_EIGHTCAP',
 'bid_price': '1.12902',
 'ask_price': '1.12902',
 'bid_size': '1000000',
 'ask_size': '1000000',
 'ts_event': 1641357702286000000,
 'ts_init': 1641357702286000000}

In [6]:
print(catalog.instruments())
print(catalog.list_data_types())


[CurrencyPair(id=EURUSD.SIM_EIGHTCAP, raw_symbol=EURUSD, asset_class=FX, instrument_class=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)]
['.DS_Store', 'bar', 'currency_pair', 'quote_tick']


In [7]:
# catalog.bars(instrument_ids=[instrument_id], start=start, end=end)

In [8]:
l, m = int(len(ticks)*0.5) , int(len(ticks)*0.5) 

In [10]:
df = pd.DataFrame([QuoteTick.to_dict(t) for t in ticks[:l]])

In [11]:
df.dtypes

type             object
instrument_id    object
bid_price        object
ask_price        object
bid_size         object
ask_size         object
ts_event          int64
ts_init           int64
dtype: object

In [12]:
# prepare dataframe from bars
def ticks_to_df(ticks):
    return pd.DataFrame([QuoteTick.to_dict(t) for t in ticks])

def ticksdf_to_barsdf(ticksdf, bar_type='15min'):
    df = ticksdf.copy()
    df['ts'] = pd.to_datetime(df.ts_event, unit='ns')
    df.bid_price = pd.to_numeric(df.bid_price)
    df.ask_price = pd.to_numeric(df.ask_price)
    df.ask_size = pd.to_numeric(df.ask_size)
    df.bid_size = pd.to_numeric(df.bid_size)

    bars = df.resample(bar_type, on='ts').agg({
        'bid_price': 'ohlc',
        'ask_price': 'ohlc',
        'ts_event': 'last',
    }).copy()

    bars.columns = ['_'.join(col).strip() for col in bars.columns.values]
    return bars




In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hvplot.pandas

In [14]:
bars = ticksdf_to_barsdf(df, bar_type='15min')

In [15]:
bars

,bid_price_open,bid_price_high,bid_price_low,bid_price_close,ask_price_open,ask_price_high,ask_price_low,ask_price_close,ts_event_ts_event
ts,,,,,,,,,
2022-01-03 00:00:00,1.13711,1.13736,1.13653,1.13723,1.13731,1.13786,1.13685,1.13776,1.641169e+18
2022-01-03 00:15:00,1.13723,1.13723,1.13715,1.13715,1.13776,1.13787,1.13747,1.13778,1.641170e+18
2022-01-03 00:30:00,1.13715,1.13741,1.13714,1.13740,1.13786,1.13787,1.13768,1.13771,1.641171e+18
2022-01-03 00:45:00,1.13740,1.13740,1.13728,1.13732,1.13768,1.13771,1.13763,1.13763,1.641172e+18
2022-01-03 01:00:00,1.13731,1.13789,1.13718,1.13733,1.13764,1.13807,1.13732,1.13734,1.641172e+18
...,...,...,...,...,...,...,...,...,...
2022-01-18 15:30:00,1.13665,1.13805,1.13662,1.13772,1.13666,1.13805,1.13662,1.13774,1.642521e+18
2022-01-18 15:45:00,1.13773,1.13802,1.13745,1.13782,1.13774,1.13802,1.13745,1.13782,1.642522e+18
2022-01-18 16:00:00,1.13783,1.13788,1.13703,1.13706,1.13782,1.13788,1.13703,1.13706,1.642522e+18


In [16]:
x_delta: pd.Timedelta = bars.index[-1] - bars.index[0]
x_delta.days

15

In [17]:
# add bollingerbands

def bollinger_bands(df, window=20, num_std=2):
    rolling_mean = df['bid_price_close'].rolling(window=window).mean()
    rolling_std = df['bid_price_close'].rolling(window=window).std()
    df['bollinger_high'] = rolling_mean + (rolling_std * num_std)
    df['bollinger_low'] = rolling_mean - (rolling_std * num_std)
    return df


In [18]:
plot_features = ['bid_price_close', 'bollinger_high', 'bollinger_low']

df = bollinger_bands(bars)


p = df.hvplot.line(x='ts', y=plot_features)
p

:NdOverlay   [Variable]
   :Curve   [ts]   (value)

In [ ]:
# use the BaseStrategy to create a simple MA cross



In [36]:
#print(f'inserting {len(ticks[l:])} ticks')

m = 1000

for i,t in enumerate(ticks[l:l+m]):        
    t_df = ticks_to_df([t])
    n = len(df)

    #print(f'inserting {t}')
    #df.loc[n] = t_data
    pd.merge(df, t_df, left_index=True, right_index=True)


print(f'{m} inserted')


ts = []
for i,t in enumerate(ticks[l:l+m]):        
    t_df = ticks_to_df([t])
    n = len(df)

    #print(f'inserting {t}')
    #df.loc[n] = t_data
    ts.append(t_df)


print(f'{m} inserted')



1000 inserted
1000 inserted


In [ ]:
expl = bars.hvplot.explorer()
expl

In [ ]:
df.hvplot.explorer()

In [ ]:
VIEW_WINDOW = 20000

In [ ]:
df[-VIEW_WINDOW:].hvplot(
    kind='line',
    x='ts',
    y=['bid_price', 'ask_price'],
    legend='bottom_right',
    widget_location='bottom',
)

In [ ]:
bars.hvplot(
    kind='step',
    x='ts',
    y=['bid_price_close', 'ask_price_close'],
    legend='bottom_right',
    widget_location='bottom',
)

In [ ]:
# Example update

import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from matplotlib.widgets import Button

# Sample data generation
np.random.seed(0)  # For reproducible results
dates = pd.date_range('20230101', periods=60)  # 60 days of data
actual_prices = np.random.randn(60).cumsum() + 100  # Simulate CFD prices
predictions = actual_prices + np.random.randn(60) * 2  # Simulate predictions with some noise

df = pd.DataFrame({'Date': dates, 'Actual_Price': actual_prices, 'Prediction': predictions})

# Initial plot setup
fig, ax = plt.subplots(figsize=(15, 7))
plt.subplots_adjust(bottom=0.2)
line1, = ax.plot(df['Date'][0:1], df['Actual_Price'][0:1], label='Actual Price', marker='o')
line2, = ax.plot(df['Date'][0:1], df['Prediction'][0:1], label='Model Prediction', linestyle='--', marker='x')
plt.title('CFD Prices and Model Predictions Over Time')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.xticks(rotation=45)

# Current index tracker
current_index = [0]

# Button event handler
def advance(event):
    current_index[0] += 1
    if current_index[0] >= len(df):
        print("Reached the end of the data.")
        return
    
    new_date = df['Date'][0:current_index[0]]
    new_actual = df['Actual_Price'][0:current_index[0]]
    new_prediction = df['Prediction'][0:current_index[0]]
    
    line1.set_data(new_date, new_actual)
    line2.set_data(new_date, new_prediction)
    ax.relim()  # Recalculate limits
    ax.autoscale_view(True,True,True)  # Rescale the view
    fig.canvas.draw()  # Redraw the figure

# Button
button_ax = plt.axes([0.81, 0.05, 0.1, 0.075])  # Adjust these parameters to place the button
button = Button(button_ax, 'Next', color='lightgoldenrodyellow', hovercolor='0.975')
button.on_clicked(advance)

plt.show()

In [ ]:
# python script version

import threading
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

# Initialize DataFrame to hold bar data
df_bars: pd.DataFrame = pd.DataFrame(columns=['timestamp', 'open', 'high', 'low', 'close'])

# Initialize the plot
plt.ion()
fig, ax = plt.subplots(figsize=(10, 5))

def generate_random_bar(timestamp):
    """Generates random bar data."""
    open_price = np.random.randint(100, 105)
    close_price = open_price + np.random.randint(-5, 5)
    high_price = max(open_price, close_price) + np.random.randint(0, 5)
    low_price = min(open_price, close_price) - np.random.randint(0, 5)
    return {'timestamp': timestamp, 'open': open_price, 'high': high_price, 'low': low_price, 'close': close_price}

def on_bar(bar):
    """Handles new bar data."""
    global df_bars
    df_bars.loc[len(df_bars)] = bar
    # Here, you could also implement logic to trigger trading decisions based on the new bar

def update_plot():
    """Updates the plot with the latest bar data."""
    while True:
        ax.clear()
        if not df_bars.empty:
            ax.plot(df_bars['timestamp'], df_bars['close'], label='Close Price')
            ax.set_title('Live Bar Data')
            ax.set_xlabel('Timestamp')
            ax.set_ylabel('Price')
            ax.legend()
            plt.xticks(rotation=45)
            plt.tight_layout()
        plt.pause(0.5)  # Adjust this to change how fast the plot updates

if __name__ == "__main__":
    # Start the plot updater in a separate thread
    threading.Thread(target=update_plot, daemon=True).start()

    start_time = datetime.now()
    for i in range(60):  # Simulate 60 bar updates
        # Generate new bar for the current timestamp
        new_bar = generate_random_bar(start_time + timedelta(minutes=i))
        on_bar(new_bar)
        time.sleep(0.1)  # Simulate waiting for new bar data


In [ ]:
# notebook version 
# no seperate thread

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

%matplotlib inline

# Initialize DataFrame to hold bar data
df_bars = pd.DataFrame(columns=['timestamp', 'open', 'high', 'low', 'close'])

def generate_random_bar(timestamp):
    """Generates random bar data."""
    open_price = np.random.randint(100, 105)
    close_price = open_price + np.random.randint(-5, 5)
    high_price = max(open_price, close_price) + np.random.randint(0, 5)
    low_price = min(open_price, close_price) - np.random.randint(0, 5)
    return {'timestamp': timestamp, 'open': open_price, 'high': high_price, 'low': low_price, 'close': close_price}

def update_plot(df_bars):
    """Redraws the plot with the latest bar data."""
    plt.figure(figsize=(10, 5))
    plt.plot(df_bars['timestamp'], df_bars['close'], label='Close Price')
    plt.title('Live Bar Data')
    plt.xlabel('Timestamp')
    plt.ylabel('Price')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    display(plt.gcf())
    clear_output(wait=True)

start_time = datetime.now()
for i in range(60):  # Simulate 60 bar updates
    new_bar = generate_random_bar(start_time + timedelta(minutes=i))
    df_bars.loc[len(df_bars)] = new_bar
    update_plot(df_bars)
    plt.pause(0.1)  # Adjust this to change how fast the plot updates
